### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현
> 이 문제는 LangChain의 Tool Calling 기능을 학습하기 위한 기초 단계입니다. 카페 메뉴 정보를 제공하는 AI 어시스턴트를 구현하면서 다양한 데이터 소스(로컬 DB, 웹, 위키피디아)에서 정보를 검색하는 방법을 익힙니다.

준비사항:
- cafe_menu.txt 파일을 ./data/ 폴더에 저장
- 필요한 라이브러리 설치: pip install langchain langchain-openai langchain-community langchain-ollama faiss-cpu
- Ollama 설치 및 bge-m3 모델 다운로드
- ollama pull bge-m3
- 환경변수 설정 (OpenAI API 키, Tavily API 키)
- 코드 실행하여 벡터 DB 생성 및 테스트

세부사항:
- Tool 정의 방법 이해: @tool 데코레이터를 사용한 사용자 정의 도구 생성
- 벡터 DB 구축: 텍스트 데이터를 임베딩하여 검색 가능한 형태로 저장
- 다중 도구 활용: 서로 다른 용도의 도구들을 하나의 LLM에 연결
- 기본 체인 구성: 도구 호출 결과를 처리하는 간단한 워크플로우 구현

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document
import re

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Google Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)

In [4]:
# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

In [5]:
# Text Split (문서 분할)
def split_cafe_menu(document):
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

menu_documents=[]
for doc in documents:
    menu_documents += split_cafe_menu(doc)
    
# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...

메뉴 번호: 2
메뉴 이름: 카페라떼
내용:
2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다...


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# Embedding - FAISS를 사용한 벡터 인덱스 생성
embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장
menu_db.save_local("./db/cafe_db")

### 3개의 도구를 정의하고 LLM에 바인딩

In [7]:
# tavily_search_func
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# 입력: 검색 쿼리 (str)
# 출력: 웹 검색 결과 (str)

@tool
def tavily_search_func(query: str) -> str: 
    """Searches the internet for information that does not exist in the database or for the latest information."""
    
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])
    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [8]:
# wiki_summary
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader
from textwrap import dedent
from pydantic import BaseModel, Field

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")


wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

C:\Users\sogno\AppData\Local\Temp\ipykernel_20848\539443416.py:37: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = summary_chain.as_tool(


In [9]:
# db_search_cafe_func
from langchain_core.documents import Document
from typing import List

# cafe db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = cafe_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

### LLM 바인딩: bind_tools() 메서드로 3개 도구를 모두 연결

In [10]:
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

In [11]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from pprint import pprint

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)           

    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [12]:
response = cafe_menu_chain.invoke("아메리카노 가격은?")
# 응답 출력 
print(response.content)

아메리카노와 아이스 아메리카노 모두 4,500원입니다.


In [13]:
response = cafe_menu_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")
# 응답 출력 
pprint(response.content)

('아메리카노는 ₩4,500이며, 주요 원료는 에스프레소와 뜨거운 물입니다. 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, '
 '원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 아이스 아메리카노 또한 ₩4,500이며, 주요 원료는 '
 '에스프레소, 차가운 물, 얼음입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다.')


### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트

In [14]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [15]:
# 카페 메뉴(카푸치노)에 맞게 프롬프트 예시를 구성합니다.

examples = [
    HumanMessage("카푸치노의 가격과 특징을 알려주세요.", name="example_user"),
    AIMessage("카푸치노 메뉴 정보를 데이터베이스에서 검색하겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "카푸치노"}, "id": "1"}]),
    ToolMessage("카푸치노: 가격 ₩5,000, 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.", tool_call_id="1"),
    AIMessage("카푸치노의 가격은 ₩5,000이며, 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다. 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "카푸치노"}, "id": "2"}]),
    ToolMessage("카푸치노는 에스프레소에 뜨거운 우유와 우유 거품을 더한 이탈리아식 커피 음료입니다. 이름은 카푸친 수도사의 갈색 옷에서 유래되었습니다.", tool_call_id="2"),
    AIMessage("카푸치노는 에스프레소에 뜨거운 우유와 우유 거품을 더한 이탈리아식 커피 음료로, 부드러운 맛과 풍부한 거품이 특징입니다.", name="example_assistant"),
]

system = """You are an AI assistant providing cafe menu information.
For information about the cafe's menu, use the search_cafe tool.
For other general information, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

In [16]:
# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

In [17]:
# Google Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)

In [18]:
# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

In [19]:
# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    
    for tool_call in ai_msg.tool_calls:

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)         

    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [22]:
# 체인 실행
query = "카푸치노와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = cafe_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

C:\Users\sogno\AppData\Local\Temp\ipykernel_20848\2880866804.py:12: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)


('카푸치노와 어울리는 디저트는 검색 결과에서 찾을 수 없었습니다. 하지만 일반적으로 카푸치노는 달콤한 디저트와 잘 어울립니다. 예를 들어, '
 '케이크, 쿠키, 타르트 등이 좋은 선택이 될 수 있습니다. 특히 초콜릿 케이크나 치즈 케이크는 카푸치노의 풍부한 맛과 잘 어울립니다.\n'
 '\n'
 '라떼는 오스트리아에서 유래되어 이탈리아에서 개발되었으며, 카푸치노와 들어가는 재료는 같지만 비율의 차이가 있습니다. 라떼는 에스프레소에 '
 '우유를 더한 음료로, 카푸치노보다 우유의 비율이 높습니다.')
